In [46]:
#setup stuff, just run this jaunt once
!pip install pandas fuzzywuzzy python-Levenshtein --quiet
import pandas as pd, json, re
from fuzzywuzzy import process
print("Setup complete")

Setup complete


In [47]:
#Read in the data 
df = pd.read_csv('fra_cleaned.csv', sep=';', encoding = 'latin1')
df = df.drop(columns=['url'])
df.head()


,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,accento-overdose-pride-edition,xerjoff,Italy,unisex,"1,42",201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",unknown,NaN,rose,woody,fruity,aromatic,floral
1,classique-pride-2024,jean-paul-gaultier,France,women,"1,86",70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,NaN,citrus,white floral,sweet,fresh,musky
2,classique-pride-2023,jean-paul-gaultier,France,unisex,"1,91",285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia-cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,musky
3,pride-edition-man,bruno-banani,Germany,men,"1,92",59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,NaN,fruity,nutty,woody,tropical,NaN
4,le-male-pride-collector,jean-paul-gaultier,France,men,"1,93",632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,NaN,aromatic,warm spicy,fresh spicy,cinnamon,vanilla


In [48]:
'''
Redditors organized the fragrantica accords in these 10 categories:

Citrus–Green
Floral–Powdery
Woody–Earthy
Amber–Resinous
Sweet–Gourmand
Spicy–Aromatic
Smoky–Leathery
Marine–Ozonic
Musky–Soapy
Fruity–Tropical

You can edit the map if you disagree, and label your changes. Make sure the case is exactly the same as above

'''

MAP = {
    # ---------- LEVEL 1 ----------
    'aldehydic': 'Citrus–Green',
    'amber': 'Amber–Resinous',
    'animalic': 'Musky–Soapy',
    'aquatic': 'Marine–Ozonic',
    'aromatic': 'Spicy–Aromatic',
    'citrus': 'Citrus–Green',
    'earthy': 'Woody–Earthy',
    'floral': 'Floral–Powdery',
    'fresh': 'Citrus–Green',
    'fresh spicy': 'Spicy–Aromatic',
    'fruity': 'Fruity–Tropical',
    'green': 'Citrus–Green',
    'herbal': 'Spicy–Aromatic',
    'lactonic': 'Sweet–Gourmand',
    'leather': 'Smoky–Leathery',
    'marine': 'Marine–Ozonic',
    'mossy': 'Woody–Earthy',
    'musky': 'Musky–Soapy',
    'oud': 'Woody–Earthy',
    'ozonic': 'Marine–Ozonic',
    'patchouli': 'Woody–Earthy',
    'powdery': 'Floral–Powdery',
    'rose': 'Floral–Powdery',
    'salty': 'Marine–Ozonic',
    'smoky': 'Smoky–Leathery',
    'soft spicy': 'Spicy–Aromatic',
    'sweet': 'Sweet–Gourmand',
    'tobacco': 'Smoky–Leathery',
    'tropical': 'Fruity–Tropical',
    'vanilla': 'Sweet–Gourmand',
    'warm spicy': 'Spicy–Aromatic',
    'white floral': 'Floral–Powdery',
    'woody': 'Woody–Earthy',
    'yellow floral': 'Floral–Powdery',

    # ---------- LEVEL 2 ----------
    'almond': 'Sweet–Gourmand',
    'balsamic': 'Amber–Resinous',
    'beeswax': 'Sweet–Gourmand',
    'bitter': 'Citrus–Green',
    'cacao': 'Sweet–Gourmand',
    'camphor': 'Spicy–Aromatic',
    'cannabis': 'Spicy–Aromatic',
    'caramel': 'Sweet–Gourmand',
    'cherry': 'Fruity–Tropical',
    'chocolate': 'Sweet–Gourmand',
    'cinnamon': 'Spicy–Aromatic',
    'coca-cola': 'Citrus–Green',
    'coconut': 'Fruity–Tropical',
    'coffee': 'Sweet–Gourmand',
    'conifer': 'Woody–Earthy',
    'honey': 'Sweet–Gourmand',
    'iris': 'Floral–Powdery',
    'lavender': 'Spicy–Aromatic',
    'metallic': 'Marine–Ozonic',
    'mineral': 'Marine–Ozonic',
    'nutty': 'Sweet–Gourmand',
    'rum': 'Smoky–Leathery',
    'sand': 'Marine–Ozonic',
    'soapy': 'Musky–Soapy',
    'sour': 'Citrus–Green',
    'tuberose': 'Floral–Powdery',
    'violet': 'Floral–Powdery',
    'whiskey': 'Smoky–Leathery',

    # ---------- LEVEL 2.5 ---------- We can keep some of these, but I don't think we can capture the scent with a generalized accord.
    'alcohol': 'Citrus–Green',
    'anis': 'Spicy–Aromatic',
    'champagne': 'Citrus–Green',
    'clay': 'Woody–Earthy',
    'oily': 'Marine–Ozonic',
    'paper': 'Woody–Earthy',
    'sake': 'Citrus–Green',
    'savory': 'Spicy–Aromatic',
    'vodka': 'Spicy–Aromatic', #Used to be Citrus-Green, but vodka just seems more masculine.
    'wine': 'Fruity–Tropical',

    # ---------- LEVEL 3 (weird) ---------- I don't even think we need these
    'asphalt': 'Smoky–Leathery',
    'bacon': 'Smoky–Leathery',
    'brown scotch tape': 'Smoky–Leathery',
    'gasoline': 'Smoky–Leathery',
    'hot iron': 'Smoky–Leathery',
    'industrial glue': 'Smoky–Leathery',
    'meat': 'Smoky–Leathery',
    'plastic': 'Smoky–Leathery',
    'rubber': 'Smoky–Leathery',
    'tennis ball': 'Citrus–Green',
    'vinyl': 'Smoky–Leathery',
    'wet plaster': 'Woody–Earthy'
}

FAMILIES = list(dict.fromkeys(MAP.values()))
print('Final 10 families:', FAMILIES)

Final 10 families: ['Citrus–Green', 'Amber–Resinous', 'Musky–Soapy', 'Marine–Ozonic', 'Spicy–Aromatic', 'Woody–Earthy', 'Floral–Powdery', 'Fruity–Tropical', 'Sweet–Gourmand', 'Smoky–Leathery']


In [49]:
#Used ChatGPT
def accords_to_vec(row):
    """row = list of 5 accord strings → 10-D L1-normalised vector"""
    vec = np.zeros(len(FAMILIES))
    for acc in row:
        if pd.isna(acc): continue
        fam = MAP.get(acc.lower().strip())
        if fam:
            vec[FAMILIES.index(fam)] += 1
    return vec / max(vec.sum(), 1)

In [50]:
import numpy as np

#This distributes the proportions of the accords for each fragrance. I made sure that it is nuanced. 

# ---------- 1. build keyword→family mirror of MAP ----------
kw2fam = {kw: fam for kw, fam in MAP.items()}

# ---------- 2. count real note occurrences ----------
def count_family_notes(text):
    """return 10-D raw counts from full note string"""
    cnt = np.zeros(len(FAMILIES))
    if pd.isna(text): return cnt
    for tok in re.findall(r"[a-zA-Z]+(?:-[a-zA-Z]+)*", text.lower()):
        fam = kw2fam.get(tok)
        if fam:
            cnt[FAMILIES.index(fam)] += 1
    return cnt

top_cnt   = np.vstack(df['Top'].apply(count_family_notes))
mid_cnt   = np.vstack(df['Middle'].apply(count_family_notes))
base_cnt  = np.vstack(df['Base'].apply(count_family_notes))

# layer weights you trust
w_top, w_mid, w_base = 4, 2, 4
note_vec = (w_top*top_cnt + w_mid*mid_cnt + w_base*base_cnt) / (w_top + w_mid + w_base)
note_vec = note_vec / np.maximum(note_vec.sum(axis=1, keepdims=True), 1)  # L1-normalise per perfume

# ---------- 3. blend with accord vote ----------
accord_vec = np.vstack([accords_to_vec(row) for row in df[accord_cols].values])  # old 5-vote

# tunable blend knob  0 = pure accord vote, 1 = pure note ratio
blend = 0.7
final_vec = np.round(final_vec, 2)          # 2-decimal float
final_vec = final_vec / final_vec.sum(axis=1, keepdims=True)
final_vec = np.round(final_vec, 2)          # second round after re-norm

# cast to string with exactly 2 decimals
dec2 = np.char.mod('%.2f', final_vec).astype(float)  # still numeric, but 2-dp clean
for i, fam in enumerate(FAMILIES):
    df[fam] = dec2[:, i]

print('Preview')
df[['Perfume']+FAMILIES].head()

Preview


,Perfume,Citrus–Green,Amber–Resinous,Musky–Soapy,Marine–Ozonic,Spicy–Aromatic,Woody–Earthy,Floral–Powdery,Fruity–Tropical,Sweet–Gourmand,Smoky–Leathery
0,accento-overdose-pride-edition,0.28,0.00,0.0,0.0,0.06,0.06,0.26,0.34,0.00,0.0
1,classique-pride-2024,0.40,0.00,0.2,0.0,0.00,0.00,0.20,0.00,0.20,0.0
2,classique-pride-2023,0.20,0.00,0.2,0.0,0.20,0.00,0.20,0.00,0.20,0.0
3,pride-edition-man,0.00,0.38,0.0,0.0,0.19,0.11,0.00,0.21,0.11,0.0
4,le-male-pride-collector,0.00,0.20,0.0,0.0,0.54,0.00,0.00,0.00,0.26,0.0


In [51]:
row = df.sample(1, random_state=66).iloc[0]
print(row['Perfume'])
print(row['Brand'])
print(row[FAMILIES].round(2).to_dict())

encens-flamboyant
goutal
{'Citrus–Green': 0.0, 'Amber–Resinous': 0.4, 'Musky–Soapy': 0.0, 'Marine–Ozonic': 0.0, 'Spicy–Aromatic': 0.6, 'Woody–Earthy': 0.0, 'Floral–Powdery': 0.0, 'Fruity–Tropical': 0.0, 'Sweet–Gourmand': 0.0, 'Smoky–Leathery': 0.0}


In [56]:
keep = ['Perfume','Brand','Gender','Country'] + FAMILIES
df[keep].to_csv('fra_simple_10family.csv', index=False)
print('saved → fra_simple_10family.csv')

saved → fra_simple_10family.csv


In [57]:
gg = pd.read_csv('fra_simple_10family.csv')
gg.head()

,Perfume,Brand,Gender,Country,Citrus–Green,Amber–Resinous,Musky–Soapy,Marine–Ozonic,Spicy–Aromatic,Woody–Earthy,Floral–Powdery,Fruity–Tropical,Sweet–Gourmand,Smoky–Leathery
0,accento-overdose-pride-edition,xerjoff,unisex,Italy,0.28,0.00,0.0,0.0,0.06,0.06,0.26,0.34,0.00,0.0
1,classique-pride-2024,jean-paul-gaultier,women,France,0.40,0.00,0.2,0.0,0.00,0.00,0.20,0.00,0.20,0.0
2,classique-pride-2023,jean-paul-gaultier,unisex,France,0.20,0.00,0.2,0.0,0.20,0.00,0.20,0.00,0.20,0.0
3,pride-edition-man,bruno-banani,men,Germany,0.00,0.38,0.0,0.0,0.19,0.11,0.00,0.21,0.11,0.0
4,le-male-pride-collector,jean-paul-gaultier,men,France,0.00,0.20,0.0,0.0,0.54,0.00,0.00,0.00,0.26,0.0
